# Experiments on Reinforcement Learning

## The game

Let's imagine a game in which the player has to choose one out of ten boxes. Every game one of the boxes lights up and the player gets one point if he or she chooses that box and zero points otherwise. The player can make 10 choices for each game.

In practice, the environment $\mathcal{E}$ is given by a random number generator that extracts a random number from 1 to 10 according to some probability distribution. The agent choose one box for 10 times, getting the corresponding reward at each attempt.

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '../src')
from rl_tools import *
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import random
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
from importlib import reload
import rl_tools
reload(rl_tools)
from rl_tools import *

## Random agent

Initialize environment, a random agent and the game.

In [ ]:
env = Environment()
agent = Agent(0.2,0)
game = Game()

Play 500 rounds with 10 attempts each and plot the results.

In [ ]:
n_rounds = 500
scores = []

for _ in range(n_rounds):
    scores.append(game.play_one_episode(agent, env, 1.0))
    
scores = np.array(scores)

In [ ]:
trace = go.Scatter(
    x = np.arange(1, len(scores)+1),
    y = scores,
    mode='markers'
)

layout = go.Layout(
    xaxis = dict(
        title='Episode number'
    ),
    yaxis = dict(
        title='Score'
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
print('Average score:')
print(scores.mean())
print('Standard deviation of the mean:')
print(scores.std())

## Neural network agent

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import mean_squared_error

In [ ]:
def custom_loss(Y_target, Y_pred):
    return mean_squared_error(Y_target, Y_pred)

In [ ]:
model = Sequential()
model.add(Dense(32, input_shape=(1,), activation='relu'))
model.add(Dense(10, activation='relu'))

model.compile(
    optimizer='adam',
    loss=custom_loss
)

model.summary()

In [ ]:
env = Environment()
nn_agent = Agent(0.2, model, random_only=False)
game = Game()

In [ ]:
n_rounds = 500
scores = []

for _ in tqdm(range(n_rounds)):
    scores.append(game.play_one_episode(nn_agent, env, 0.0))
    
scores = np.array(scores)

In [ ]:
trace = go.Scatter(
    x = np.arange(1, len(scores)+1),
    y = scores,
    mode='markers'
)

layout = go.Layout(
    xaxis = dict(
        title='Episode number'
    ),
    yaxis = dict(
        title='Score'
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
print('Average score:')
print(scores[350:].mean())
print('Standard deviation:')
print(scores.std())

# Building the training routine

Trining: building a training batch and computing the target variables.

In [ ]:
training_batch = np.array(
    random.sample(agent.memory, int(len(agent.memory)/10))
)
training_batch

In [ ]:
training_batch.shape

In [ ]:
training_batch[:,3].shape

In [ ]:
training_batch[:,3]

Compute the Q-values (each of which is a 10-component array) for each final state in the transitions.

In [ ]:
np.argmax(nn_agent.compute_q(training_batch[:,3]), axis=1)

Compute the target variable ($y_i$) for each of the transitions in the training batch.

In [ ]:
np.amax(nn_agent.compute_q(training_batch[:,3]), axis=1)+1

In [ ]:
Y_target = (training_batch[:,2]
    + nn_agent.gamma
    * np.amax(nn_agent.compute_q(training_batch[:,3]), axis=1))

In [ ]:
Y_target.shape

Compute the "prediction" from the model, based on on the initial state and action taken. This implies computing the Q-value (array) associated to the initial state and then selecting the component of each array according to which action the agent performed in each trainsition.

In [ ]:
Y_pred = np.take(nn_agent.compute_q(training_batch[:,0]), training_batch[:,1])

In [ ]:
Y_pred.shape

Optimization is done on the MSE between Y_pred and Y_target.

In [ ]:
nn_agent.model.fit(Y_target, Y_pred, epochs=1)